In [1]:
# standard
import pandas as pd
import numpy as np
import random
import os
import math

# tf and keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error

# plots and images
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

#silence TF
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Setting random seeds
random.seed(0)
np.random.seed(0)
tf.keras.backend.clear_session()
tf.random.set_seed(0)

#### read in data
data1 directory has dense one-hot encodings <br>
data2 directory has sparse one-hot encodings

In [2]:
# read in data
x_train = pd.read_csv("data2/X_train.csv").drop(columns=['Unnamed: 0'])
y_train = pd.read_csv("data2/Y_train.csv").drop(columns=['Unnamed: 0'])
x_val = pd.read_csv("data2/X_val.csv").drop(columns=['Unnamed: 0'])
y_val = pd.read_csv("data2/Y_val.csv").drop(columns=['Unnamed: 0'])
x_test = pd.read_csv("data2/X_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv("data2/Y_test.csv").drop(columns=['Unnamed: 0'])

#### Baseline

In [3]:
# calculate baseline
baseline = y_train.total_points.mean()

# compute RMSE of baseline on test and train data
mse_train = np.square(np.subtract(y_train.total_points , baseline)).mean()
mse_test = np.square(np.subtract(y_test.total_points , baseline)).mean()

print(f"train MSE: {mse_train}")
print(f"test MSE: {mse_test}")

train MSE: 6.302880420704537
test MSE: 5.573325945935219


#### KNN Model

In [11]:
# Column combination possibilities

all_cols = x_train.columns

lag_1 = ['assists_lag_1', 'bonus_lag_1', 'bps_lag_1', 'clean_sheets_lag_1',
       'creativity_lag_1', 'goals_conceded_lag_1', 'goals_scored_lag_1',
       'ict_index_lag_1', 'influence_lag_1', 'minutes_lag_1',
       'own_goals_lag_1', 'penalties_missed_lag_1', 'penalties_saved_lag_1',
       'red_cards_lag_1', 'saves_lag_1', 'selected_lag_1',
       'player_team_score_lag_1', 'opp_team_score_lag_1', 'threat_lag_1',
       'total_points_lag_1', 'value_lag_1', 'yellow_cards_lag_1']

lag_3 = ['assists_lag_3', 'bonus_lag_3', 'bps_lag_3', 'clean_sheets_lag_3',
       'creativity_lag_3', 'goals_conceded_lag_3', 'goals_scored_lag_3',
       'ict_index_lag_3', 'influence_lag_3', 'minutes_lag_3',
       'own_goals_lag_3', 'penalties_missed_lag_3', 'penalties_saved_lag_3',
       'red_cards_lag_3', 'saves_lag_3', 'selected_lag_3',
       'player_team_score_lag_3', 'opp_team_score_lag_3', 'threat_lag_3',
       'total_points_lag_3', 'value_lag_3', 'yellow_cards_lag_3']

lag_5 = ['assists_lag_5', 'bonus_lag_5', 'bps_lag_5', 'clean_sheets_lag_5',
       'creativity_lag_5', 'goals_conceded_lag_5', 'goals_scored_lag_5',
       'ict_index_lag_5', 'influence_lag_5', 'minutes_lag_5',
       'own_goals_lag_5', 'penalties_missed_lag_5', 'penalties_saved_lag_5',
       'red_cards_lag_5', 'saves_lag_5', 'selected_lag_5',
       'player_team_score_lag_5', 'opp_team_score_lag_5', 'threat_lag_5',
       'total_points_lag_5', 'value_lag_5', 'yellow_cards_lag_5']

cat = ['season', 'player_team_name', 'opp_team_name',
       'opp_diff_ind', 'position', 'GW', 'was_home']

judgmental_imp = ['player_team_name_mapping', 'opp_team_name_mapping',
       'opp_diff_ind_mapping', 'position_mapping', 'was_home_mapping',
       'assists_lag_3', 'bonus_lag_3', 'goals_scored_lag_3', 'ict_index_lag_3',
       'minutes_lag_3', 'red_cards_lag_3', 'selected_lag_3', 'player_team_score_lag_3',
       'opp_team_score_lag_3', 'total_points_lag_3', 'value_lag_3', 'yellow_cards_lag_3'] # Uses lag 3 as a default

In [ ]:
cat_columns = ['season', 'player_team_name', 'opp_team_name',
       'opp_diff_ind', 'position', 'GW', 'was_home']
df_encoded = pd.get_dummies(x_train, columns = cat_columns)

#### run KNN model using CV to choose best K

In [56]:
def run_knn(df_x_train, df_x_test, cv=10):
    
    # running for different K values to know which yields the max accuracy.
    scores=[]
    for K in range(20):   
        K = K+1
        clf = KNeighborsRegressor(n_neighbors = K)
        clf.fit(df_x_train, y_train)
        score = cross_val_score(clf, df_x_train, y_train, cv=10)
        scores.append(score.mean())
    
    mse = [1-x for x in scores]
    optimal_k = mse.index(min(mse)) + 1
    print("Optimal K: ", optimal_k)
    
    # Training the model on Optimal K.
    clf_optimal = KNeighborsRegressor(n_neighbors = optimal_k)
    clf_optimal.fit(df_x_train, y_train)
    y_pred = clf_optimal.predict(df_x_test)
    print("MSE Error: ", mean_squared_error(y_test, y_pred))

#### this is ideal model, adding lagged variables with the categorical variables
however, right now, it's throwing an error because there's categorical variables in the test data that aren't seen in the training data...

In [ ]:
cat_columns = ['season', 'player_team_name', 'opp_team_name',
       'opp_diff_ind', 'position', 'GW', 'was_home']

df_xtrain_1 = x_train[lag_1 + cat]
df_xtest_1 = x_test[lag_1 + cat]
df_xtrain_1_encoded = pd.get_dummies(df_xtrain_1, columns = cat_columns)
df_xtest_1_encoded = pd.get_dummies(df_xtest_1, columns = cat_columns)
run_knn(df_xtrain_1, df_xtest_1, cv=10)

#### lag1 model

In [52]:
df_xtrain_1 = x_train[lag_1]
df_xtest_1 = x_test[lag_1]
run_knn(df_xtrain_1, df_xtest_1, cv=10)

Optimal K:  19
RMS Error:  4.155893859988786


#### lag3 model

In [53]:
df_xtrain_3 = x_train[lag_3]
df_xtest_3 = x_test[lag_3]
run_knn(df_xtrain_3, df_xtest_3, cv=10)

Optimal K:  19
RMS Error:  4.136794970478052


#### lag5 model

In [54]:
df_xtrain_5 = x_train[lag_5]
df_xtest_5 = x_test[lag_5]
run_knn(df_xtrain_5, df_xtest_5, cv=10)

Optimal K:  19
RMS Error:  4.247562342546564


#### all the data

In [58]:
df_xtrain_all = x_train[lag_1 + lag_3 + lag_5]
df_xtest_all = x_test[lag_1 + lag_3 + lag_5]
run_knn(df_xtrain_all, df_xtest_all, cv=10)

Optimal K:  20
MSE Error:  4.072635068854933


#### OLD: run KNN model for lag3 data 

In [ ]:
mse_val = [] #to store mse values for different k
for K in range(20):
    K = K+1
    model = KNeighborsRegressor(n_neighbors = K)
    model.fit(x_train, y_train)  #fit the model
    pred=model.predict(x_test) #make prediction on test set
    error = mean_squared_error(y_test,pred) #calculate rmse
    mse_val.append(error) #store mse values
    
min_mse_index = mse_val.index(min(mse_val))
min_mse = min(mse_val)
print('Min mse: k=', min_mse_index , 'is:', min_mse)